## Creating managed tables

Since we're not specifying the `LOCATION` keyword, this table is considered managed in this database.

In [0]:
USE CATALOG hive_metastore;

CREATE TABLE managed_default (
  country STRING,
  code STRING,
  dial_code STRING
);

INSERT INTO managed_default VALUES
('France', 'Fr', '33')

Executing the `DESCRIBE EXTENDED` command on our table provides advanced metadata information. Focus on three key elements:
- The type of table, which is indeed `MANAGED``
- The location, which shows that our table resides in the default Hive metastore under `dbfs:/user/hive/warehouse`
- The provider, which confirms that this is a Delta Lake table

In [0]:
DESCRIBE EXTENDED managed_default

## Creating external tables

To achieve this, we simply add the `LOCATION` keyword followed by the desired storage path. In our case, we'll store this table under the `/mnt/demo` directory through the DBFS:

In [0]:
CREATE TABLE external_default (
  country STRING,
  code STRING,
  dial_code STRING
)
LOCATION 'dbfs:/mnt/demo/external_default';

INSERT INTO external_default VALUES
('France', 'Fr', '33')

In [0]:
DESCRIBE EXTENDED external_default

## Dropping tables

It is important to note thtat the behavior differs for managed and external tables. Let's discuss the consequences of this action on each table type. We start by running the `DROP TABLE` command on our managed table:

In [0]:
DROP TABLE managed_default

When drop a table, it deletes its metadata from the metastore. This means that the table's definition, including it's schema, column names, data types, and other relevatn information, is no longer store in the metastore. We can confirm this by trying to query the table, which will result in a "table not found" error:

In [0]:
SELECT * FROM managed_default

Dropping the manged table not only removes its metadata from the metastore, but also deletes all associated data files from the storage:

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/managed_default'

Dropping the external table also removes its entry from the metastore. We can confirm this by trying to query the table, which should result in a "table not found" error.

In [0]:
DROP TABLE external_default

In [0]:
SELECT * FROM external_default

However, the underlying data is stored outside the database directory, the data files remain intact. We can easily confirm that the data files of the table still persist by checking the table directory:

In [0]:
%fs ls 'dbfs:/mnt/demo/external_default'

In Datarbricks, is it possible to access a Delta table by querying its directory:

In [0]:
SELECT * FROM DELTA.`dbfs:/mnt/demo/external_default`

Is it possile to remove the table directory and its content by the running the `dbutils.fs.rm` function in Python:

In [0]:
%python
dbutils.fs.rm('dbfs:/mnt/demo/external_default', recurse=True)

## Working with new schema

We can create additional databases and manage tables within those databaes.

### Creating a new database

`CREATE SCHEMA` or `CREATE DATABASE` are interchangeable:

In [0]:
CREATE SCHEMA new_default

In [0]:
DESCRIBE DATABASE EXTENDED new_default

The new database is store under the default Hive directory with a `.db` extension to distinguished it from other table folders in the directory.

### Creating new tables in the new database

In [0]:
USE DATABASE new_default;

-- create managed table
CREATE TABLE managed_new_default (
  country STRING,
  code STRING,
  dial_code STRING
);

INSERT INTO managed_new_default
VALUES ('France', 'Fr', '+33');

--------------------------------
-- create and external table
CREATE TABLE external_new_default (
  country STRING,
  code STRING,
  dial_code STRING
)
LOCATION 'dbfs:/mnt/demo/external_new_default';

INSERT INTO external_new_default
VALUES ('France', 'Fr', '+33');

By running `DESCRIBE EXTENDED` on each of these tables, we can see that the first table is indeed a manged table created in its sdatabase folder inder the defait Hive directory.

In [0]:
DESCRIBE EXTENDED managed_new_default

Meanwhile, the second table, where we use the `LOCATION` keyword, has been defined as an external table under the `/mnt/demo` location.

In [0]:
DESCRIBE EXTENDED external_new_default

### Dropping tables

Dropping the tables removes their entrie form the Hive metastore. You can easily confirm this in the Catalog Explorer.

In [0]:
DROP TABLE managed_new_default;
DROP TABLE external_new_default;

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/new_default.db/managed_new_default'

In [0]:
%fs ls 'dbfs:/mnt/demo/external_new_default'

## Working in a Custom-Location Schema

### Creating the database

To achieve this, we begin by using the `CREATE SCHEMA` statement, and we add the `LOCATION` keyword followed by the desired storage path:

In [0]:
CREATE SCHEMA custom
LOCATION 'dbfs:/Shared/schemas/custom.db'

In [0]:
DESCRIBE DATABASE EXTENDED custom

### Creating tables

In [0]:
USE DATABASE custom;

-- create managed table
CREATE TABLE managed_custom (
  country STRING,
  code STRING,
  dial_code STRING
);

INSERT INTO managed_custom
VALUES ('France', 'Fr', '+33');

--------------------------------
-- create and external table
CREATE TABLE external_custom (
  country STRING,
  code STRING,
  dial_code STRING
)
LOCATION 'dbfs:/mnt/demo/external_custom';

INSERT INTO external_custom
VALUES ('France', 'Fr', '+33');

In [0]:
DESCRIBE EXTENDED managed_custom

In [0]:
DESCRIBE EXTENDED external_custom

### Dropping tables

In [0]:
DROP TABLE managed_custom

In [0]:
DROP TABLE external_custom